# PROBABILITY MATCHING, PART 3a

### Recapping the mathematical model approach

Gallistel (1990) makes a compelling case that we *can* model probability matching in groups of foraging animals by assuming each animal represents the full complexity of the situation. That is, on Gallistel's *representational* theory, each duck (in our example domain) respresents rate and magnitude of food available at each source and integrates those over time to assess the prevalence of food (size units per unit of time, e.g., $grams/minute$, although we do not assume the ducks have such specific units; in other parts of the book, Gallistel elegantly delves into how anmimals can represent time, rate, and magnitude [among other things] in a variety of domains). Once a duck has calculated (estimated) prevalence at each source, it needs to evaluate which source is preferable. A simple way to do this is simply consider the ratio of each source to the sum of both sources, e.g.: 

$$
preference = \frac{prevalence_{max}}{prevalence_1 + prevalence_2}
$$ 

Where $prevalence_{max}$ is the larger $prevalence$, and this ratio tells us how much more preferable the larger one is. (In the previous notebook, there was also some exponential weighting that introduces a nonlinearity, but this is not crucial.) Recall that there is a hiccup here, though: if each duck does the same calculation, each duck will go toward the food source with higher prevalence. To prevent this, a $switchiness$ variable is introduced, with the idea that ducks will vary in how willing they are to change sources if they are at the less prevalent source. 

### Towards an Agent-Based approach

Let's try modeling this system another way. Instead of devising equations that describe the system rather abstractly, let's see what it would take to simulate the moment-by-moment behavior of ducks.

To do this, we are going to use **agent-based modeling**. This has some similarity to the cellular automata we played around with using the Game of Life simulator. But instead of our primitive elements only being fixed locations on a grid (although we did see the emergence of intercellular interactions that appeared to cohere and act as units), we are going to populate a grid with *agents* (ducks) that can navigate the grid (the pond). We will also need a method for dropping food in regions of the pond. Then we have to think about how the ducks will update their representations of food prevalence at the two sources, how they should move towards food, whether/how they should move if there is no food in the environment, how quickly they can move, etc.

So let's think about all the things we have to define in order to create a simulation.

- Pond size and shape (let's keep it a rectangle, so we just have to define X and Y dimensions). Let's start with a 40x40 grid.
- Food source dimensions -- let's put them at the top and bottom of the pond, but how big should they be? Let's start with a the sources having a size based on 8: they will extend 4 cells to the left and right of the pond center, and 8 cells from the top or bottom end.
- Number of ducks -- let's start with 50.
- Rates and morsel magnitudes
- How ducks will update representations
- How many timesteps the simulation will last

Please review the code below -- at least skim it. Then try the simulator below and see if you can understand how it works. We will try a few things.

- For moving: ducks will first have to update their source preference, and then make a decision based on their personal *switchiness* value (which we'll distribute normally). They will update their preference, and then choose their food source by generating a random number. If that number is larger than their switchiness value, they will do a second decision. If a second random number is larger than the ratio of densities, they choose the denser source.
    - Then, they will 'aim' at the nearest morsel of available food at their preferred source, and move to the 'best' adjacent unoccupied cell that will move them closer to that piece of food.
    - If no food is available at their preferred source, they will move to a random adjacent unoccupied cell. If all cells are occupied, they stay where they are.
 
`Unfortunately, there is a bug in this simulator that I cannot solve. The ducks are initially blue, and food is red when it appears. If a duck reaches a morsel of food, it turns violet while it is eating (in this simulator, that is just 1 timestep). However, occasionally, *all* of the ducks turn red or violet for a timestep or two. Underlyingly, the states are all correct, which you can see if you click the 'Show text plot' button before you run a simulation. This makes a large text version of the plot and when all the ducks turn red or violet, you'll see that their states in the text plot have not changed. This seems to be a bug in the matplotlib library. Unfortunately, we just have to tolerate this for now. Sorry!` 

 

In [1]:
import numpy as np
import random
import math
from IPython.display import display, clear_output
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
import time
import threading

# Distance function
def distance(pos1, pos2):
    x1, y1 = pos1
    x2, y2 = pos2
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

# Duck class
class Duck:
    def __init__(self, unique_id, model, switchiness_mean, switchiness_sd):
        self.unique_id = unique_id
        self.model = model
        self.eating = 0
        self.pos = (random.randrange(model.width), random.randrange(model.height))
        self.switchiness = min(max(np.random.normal(switchiness_mean, switchiness_sd), 0), 1) # Trimmed between 0 and 1
        self.preference = 1 if distance(self.pos, (model.width * 0.3, model.height * 0.1)) < distance(self.pos, (model.width * 0.3, model.height * 0.8)) else 2

    def determine_preference(self):
        prev_preference = self.preference
        notzero = 0.00000001 # prevent division by zero
        if random.random() < self.switchiness:
            relative_prevalence1 = (self.model.total_food_source1 + notzero) / (self.model.total_food_source1 + self.model.total_food_source2 +  (notzero * 2))
            self.preference = 1 if random.random() < relative_prevalence1 else 2
        else:
            self.preference = prev_preference
    
    # Move function
    def move(self):
        possible_steps = [((self.pos[0] + dx) % self.model.width, 
                           (self.pos[1] + dy) % self.model.height) 
                          for dx in range(-1,2) for dy in range(-1,2)]
        possible_steps.remove(self.pos)
        
        # Get the duck-free steps.
        duck_free_steps = [step for step in possible_steps if self.model.is_duck_free(step)]
        
        if not duck_free_steps:
            return  # No place to move.
        
        if self.model.representational:
            # if representational, then we restrict potential food targets to those near the preferred source
            self.determine_preference()
            preferred_source_y_range = self.model.feeder_1_y_range if self.preference == 1 else self.model.feeder_2_y_range
            food_positions = [food.pos for food in self.model.foods if food.pos[1] in preferred_source_y_range]
        else:
            # nonpresentational: all food are potential targets, will aim for nearest uneaten food
            food_positions = [food.pos for food in self.model.foods]
        
        if food_positions:
            nearest_food = min(food_positions, key=lambda pos: distance(self.pos, pos))
            
            # Filter the possible steps, choose only those that are duck-free.
            distances = [distance(step, nearest_food) for step in duck_free_steps]
            min_distance = min(distances)
            closest_steps = [step for step, dist in zip(duck_free_steps, distances) if dist == min_distance]
            
            if closest_steps:
                self.pos = random.choice(closest_steps)
        else:
            # If no food is available, move randomly.
            self.pos = random.choice(duck_free_steps)

    # Step function
    def step(self):
        if self.eating > 0:
            self.eating -= 1
            return
        food_at_pos = [food for food in self.model.foods if food.pos == self.pos]
        if food_at_pos:
            food_to_eat = food_at_pos[0]
            self.eating = food_to_eat.size
            self.model.foods.remove(food_to_eat)
        else:
            self.move()

# Food class
class Food:
    def __init__(self, unique_id, model, size):
        time_to_consume = 1
        self.unique_id = unique_id
        self.model = model
        if food_slider.value:
            # food will take size steps to consume
            self.size = size
        else: 
            # for this version, food just takes 1 step to consume
            self.size = time_to_consume

        self.pos = (random.choice(model.feeder_x_range), 
                    random.choice(random.choice([model.feeder_1_y_range, model.feeder_2_y_range])))

# Pond class (Model)
class Pond:
    def __init__(self, width, height, xrange, yrange, N, \
                 feeder1_rate, feeder1_size, feeder2_rate, feeder2_size, \
                switchiness_mean, switchiness_sd, representational):
        self.representational = representational
        self.width = width
        self.height = height
        self.num_ducks = N
        self.speed = initial_speed # default animation speed
        # self.ducks = [Duck(i, self) for i in range(N)]
        self.ducks = [Duck(i, self, switchiness_mean, switchiness_sd) for i in range(N)]
        self.foods = []
        self.feeder1_rate = feeder1_rate
        self.feeder1_size = feeder1_size
        self.feeder2_rate = feeder2_rate
        self.feeder2_size = feeder2_size
        self.feeder_x_range = range(int((1-xrange) * self.width), int(xrange * self.width))
        self.feeder_2_y_range = range(int(yrange * self.height), int((1-xrange) * self.height))
        self.feeder_1_y_range = range(int(xrange * self.height), int((1-yrange) * self.height))

        self.total_food_source1 = 0
        self.total_food_source2 = 0

        self.timestep = 0
        self.uneaten1 = 0
        self.uneaten2 = 0
        self.crossed_midline = 0

    def is_duck_free(self, pos):
        return all(duck.pos != pos for duck in self.ducks)

    def step(self):
        self.uneaten1 = 0
        self.uneaten2 = 0
        for food in self.foods:
            if food.pos[1] in self.feeder_1_y_range:
                self.uneaten1 += food.size
            elif food.pos[1] in self.feeder_2_y_range:
                self.uneaten2 += food.size

        prev_duck_positions = [duck.pos for duck in self.ducks]
        random.shuffle(self.ducks)
        for duck in self.ducks:
            duck.step()
        
        # Check if ducks crossed the midline
        midline = self.height // 2
        self.crossed_midline = 0
        for prev_pos, duck in zip(prev_duck_positions, self.ducks):
            if (prev_pos[1] < midline and duck.pos[1] >= midline) or (prev_pos[1] >= midline and duck.pos[1] < midline):
                self.crossed_midline += 1

        if self.timestep % self.feeder1_rate == 0:
            self.drop_food(self.feeder_1_y_range, self.feeder1_size)
        if self.timestep % self.feeder2_rate == 0:
            self.drop_food(self.feeder_2_y_range, self.feeder2_size)
        self.timestep += 1
        
    def drop_food(self, y_range, size):
        food = Food(len(self.ducks) + len(self.foods), self, size)
        food.pos = (random.choice(self.feeder_x_range), random.choice(y_range))
        self.foods.append(food)
        if y_range == self.feeder_1_y_range:
            self.total_food_source1 += size
        else:
            self.total_food_source2 += size

    def plot_grid(self):
        grid = np.zeros((self.width, self.height), dtype=int)
        for food in self.foods:
            x, y = food.pos
            grid[x, y] = 3
        for duck in self.ducks:
            x, y = duck.pos
            grid[x, y] = 2 if duck.eating else 1
        return grid

# Parameters
pondwidth = 40
pondheight = 40
feeder_xrange = 0.7 # e.g., feeders use 70% of width
feeder_yrange = 0.1 # e.g., feeders use 10% of height
nducks = 50
rate1 = 4
rate2 = 8
size1 = 8
size2 = 8
timesteps = 100
switchiness_sd = 0.1
switchiness_mean = 0.5
initial_speed = 10
repnonrep = 1
foodtakestime = 0
plotsize = 20 # make this bigger to make the image of the grid larger

# def toggle_representation(button):
#     global repnonrep
#     # Assume repnonrep is a boolean: True means Representation and False means Non-Representation
#     repnonrep = not repnonrep  # Toggle the repnonrep variable

#     model.representation = repnonrep
#     # Update the button's label to reflect the current state
#     # button.description = "REP" if model.representation else "NONrep"

# rep_button = widgets.Button(description="REP/NONrep")
# rep_button.on_click(toggle_representation)

model = Pond(pondwidth, pondheight, feeder_xrange, feeder_yrange, nducks, rate1, rate2, size1, size2, \
            switchiness_mean, switchiness_sd, repnonrep)

# Plot function
def plot_grid(model):
    global plotsize
    pond_grid = model.plot_grid()
    cmap = ListedColormap(['white', 'blue', 'violet', 'red'])

    figx = int(pwidth_slider.value * (plotsize_slider.value * 0.01))
    figy = int(pheight_slider.value * (plotsize_slider.value * 0.01))
    
    plt.figure(figsize=(figx, figy))  # adjust the numbers (10, 8) as desired
    plt.imshow(pond_grid.T, cmap=cmap, origin='lower', interpolation='none')

    #plt.imshow(pond_grid.T, cmap=cmap, origin='lower', interpolation = 'none')
    plt.axhline(y=pheight_slider.value / 2, color='grey', linestyle='--')

    # Counting how many ducks are closer to feeder1 and feeder2
    # count_closer_to_feeder1 = sum(distance(duck.pos, (25, 5)) < distance(duck.pos, (25, 45)) for duck in model.ducks)
    # count_closer_to_feeder2 = model.num_ducks - count_closer_to_feeder1
    # proportion_closer_to_feeder1 = count_closer_to_feeder1 / model.num_ducks

     # Calculate the midline
    midline = pheight_slider.value // 2

    # Count the number of ducks above and below the midline
    count_closer_to_feeder2 = sum(1 for duck in model.ducks if duck.pos[1] < midline)
    count_closer_to_feeder1 = model.num_ducks - count_closer_to_feeder2

    proportion_closer_to_feeder1 = count_closer_to_feeder1 / model.num_ducks
   
    # Getting position and state of duck 1
    duck1_pos = model.ducks[0].pos
    duck1_state = "Eating" if model.ducks[0].eating > 0 else "Not Eating"
    duck1_eating = model.ducks[0].eating
    
    # Counting how many ducks are eating
    count_eating = sum(duck.eating > 0 for duck in model.ducks)
    
    # Calculating total uneaten food
    total_uneaten_food = sum(food.size for food in model.foods)
    
    # Calculating the sum of all duck states
    sum_duck_states = sum(duck.eating for duck in model.ducks)

    # concatenate duck states into a string
    duck_states_str = ''.join(str(duck.eating) for duck in model.ducks)

    # concatenate grid values -- giant string, but need for debugging color problem
    #tmpgrid = pond_grid.T
    grid_values_str = ''.join(str(val) + ('\n' if i % pondwidth == (pondwidth-1) else '') for i, val in enumerate(pond_grid.T.flatten()))
    grid_text_str = "\n" # initialize with newline because otherwise the first line is shifted because it's centered with caption
    if show_ascii_plot_button.value:
        reversed_rows = grid_values_str.strip().split('\n')[::-1]
        midline_position = pheight_slider.value // 2
        bordered_rows = ['|' + row + '|' for row in reversed_rows]
        midline_row = '|' + '-' * (pwidth_slider.value + 1) + '|'
        bordered_rows.insert(midline_position, midline_row)

        #bordered_rows = ['|' + row + '|' for row in reversed_rows]
        top_border = '+' + '-' * pwidth_slider.value + '+'
        bordered_rows = [top_border] + bordered_rows + [top_border]
        grid_text_str += '\n'.join(bordered_rows).replace('0', ' ').replace('1', 'd').replace('2','X').replace('3','#')
    
    #repnonrep = model.representational
        
    plt.title(f"Step: {model.timestep} Representational: {model.representational}\n\
                Near source 1: {count_closer_to_feeder1} = {proportion_closer_to_feeder1:.2f},\
                \nUneaten={total_uneaten_food} Eating={count_eating} State sum={sum_duck_states}\n\
                Uneaten near source1: {model.uneaten1}, Uneaten near source2: {model.uneaten2}\n\
                Ducks crossed midline: {model.crossed_midline}\n\
                Duck 1: {duck1_pos},{duck1_eating},{duck1_state}\n\
                {grid_text_str}", fontsize=10, fontname='Courier')


                    #{duck_states_str}\n\

    # Adding a light blue border around the grid
    for edge, spine in plt.gca().spines.items():
        spine.set_visible(True)
        spine.set_linewidth(2)
        spine.set_edgecolor('lightblue')

    plt.xticks([])  # Remove x-axis ticks
    plt.yticks([])  # Remove y-axis ticks
    plt.show()
        
simulating = True

def pause_simulation(b):
    global simulating
    if simulating:
        # If simulation is running, pause it
        simulating = False
        pause_button.description = "Resume"
    else:
        # If simulation is paused, resume it
        simulating = True
        pause_button.description = "Pause"
        # Restart the simulation
        start_simulation()

pause_button = widgets.Button(description="Pause")
pause_button.on_click(pause_simulation)


# def toggle_pause(_):
#     global paused
#     paused = not paused
#     pause_button.description = "Resume" if paused else "Pause"

# pause_button = widgets.Button(description="Pause")
# pause_button.on_click(toggle_pause)



current_speed = 0.1  # Declare a global variable for the current speed

# def update_speed(change):
#     global current_speed
#     current_speed = change.new  # Update the global variable
    
# Add sliders for parameters
rep_slider = widgets.IntSlider(value=repnonrep, min=0, max=1, description="Representational:")
food_slider = widgets.IntSlider(value=foodtakestime, min=0, max=1, description="Food takes time:")
plotsize_slider = widgets.IntSlider(value=plotsize, min=5, max=50, step=1, description="Plot size:")
pwidth_slider = widgets.IntSlider(value=pondwidth, min=10, max=200, description="Width:")
pheight_slider = widgets.IntSlider(value=pondheight, min=10, max=200, description="Height:")
nducks_slider = widgets.IntSlider(value=nducks, min=1, max=200, description="Ducks:")
rate1_slider = widgets.IntSlider(value=rate1, min=1, max=20, description='Rate 1:')
rate2_slider = widgets.IntSlider(value=rate2, min=1, max=20, description='Rate 2:')
size1_slider = widgets.IntSlider(value=size1, min=1, max=20, description='Size 1:')
size2_slider = widgets.IntSlider(value=size2, min=1, max=20, description='Size 2:')
speed_slider = widgets.IntSlider(value=initial_speed, min=1, max=100, description='Speed:')
#speed_slider.observe(update_speed, names='value')
timesteps_slider = widgets.IntSlider(value=timesteps, min=1, max=1000, description='Timesteps:')
show_ascii_plot_button = widgets.ToggleButton(value=False, description='Show text Plot', button_style='')



# Define the plot_output widget
plot_output = widgets.Output()
#plot_output.layout.height = '1500px'  # for example, to set it to 500 pixels


def run_simulation():
    global model
    nducks = nducks_slider.value
    repnonrep = rep_slider.value
    pondwidth = pwidth_slider.value
    pondheight = pheight_slider.value
    model = Pond(pondwidth, pondheight, feeder_xrange, feeder_yrange, nducks, rate1, rate2, size1, size2, switchiness_mean, switchiness_sd, repnonrep)
    model.timestep = 0  # Resetting the timestep
    timesteps_from_slider = timesteps_slider.value  # Get the value from the slider
    for _ in range(timesteps_from_slider):
        while not simulating:  # Keep checking if the simulation is paused
            time.sleep(0.1)  # Sleep to prevent high CPU usage

        model.feeder1_rate = rate1_slider.value
        model.feeder2_rate = rate2_slider.value
        model.feeder1_size = size1_slider.value
        model.feeder2_size = size2_slider.value
        model.representational = rep_slider.value

        # Sleep to control animation speed
        time.sleep(speed_slider.value * 0.01)
        
        model.step()
        with plot_output:
            plot_grid(model)
            clear_output(wait=True)
        
def start_simulation(change):
    run_simulation()

    
def start_simulation_thread(change):
    simulation_thread = threading.Thread(target=run_simulation)
    simulation_thread.start()

start_button = widgets.Button(description="Start")
start_button.on_click(start_simulation)
# Put the buttons in a horizontal box (HBox)
buttons = widgets.HBox([start_button, show_ascii_plot_button])
# Group sliders in horizontal boxes
dim_sliders = widgets.HBox([pwidth_slider, pheight_slider, plotsize_slider])
rate_sliders = widgets.HBox([rate1_slider, rate2_slider])
size_sliders = widgets.HBox([size1_slider, size2_slider])
timey_sliders = widgets.HBox([nducks_slider, timesteps_slider, speed_slider])
misc_sliders = widgets.HBox([rep_slider, food_slider])


vbox = widgets.VBox([buttons, misc_sliders, dim_sliders, rate_sliders, size_sliders, timey_sliders, plot_output])
#vbox.layout.height = '200px'
display(vbox)


# Lab report, part 1

First, just explore the simulator. You can control various parameters with the sliders. You may find it helpful to set `Timesteps` to a lower value as you explore parameters. Currently, none of the sliders take effect *during* a simulation: they take effect when you start the next simulation. *For now, don't use the `Representational` slider or the 'Food takes time' slider.*

1. Try a few different rate combinations, keeping both sizes 8. What happens if you set both rates to the same small value, medium value, or large value? What happens if you make the ratio 2:1 (rate1 twice as large as rate2)?
2. Try a few different size combinations, initially setting both rates to 4. Explore and report a few observations.
3. With one interesting combination you found for #1 or #2 (or one or a few from each), see what happens as you make the pond taller. You may find the `Plot size` slider helpful to make the grid fit nicely in your browser window.
4. Draw some general conclusions. This simulation is using the 'representational' approach, where each duck tracks food density over time (size units per time units). Is it able to simulate probability matching due to differences in rate? Is it able to simulate probability matching due to differences in size?

# Lab report, part 2

We saw that with the 'representational' approach, we seem to be able to do a fine job of simulating probability matching to prevalence differences due to rate or magnitude. Gallistel makes a very bold claim about this approach: 

> [Results suggest] that birds accurately represent rates, that they accurately represent morsel magnitudes, and that they can multiply the representation of morsels per unit time by the representation of morsel magnitude to compute the internal variables that determine the relative likelihood of their choosing one foraging patch over the other. (p. 358)

He makes an even bolder claim for alternative theories:

> The challenge for nonrepresentational theories ... Is to propound a nonrepresentational model of ... a system that is altered by its past experiences in such a way that it chooses patches in proportion to their relative [densities] ... from ... observations alone ... without the system's having any internal variable[s] ... or a fortiori any operations that appropriately combine these nonexistent representatives of number, time, and magnitude. Is it possible to propound a model of the internal causation of the duck's behavior that avoids postulating an isomorphism between a system of variables and operations inside the duck and the corresponding system of number, time and magnitude external to the duck? (p. 359)

As some of you have noted, his idea of a 'nonrepresentational' account still seems to require a fair amount of representation -- some way of learning which source is 'better' without explicitly tracking rate and size. However, can we make a more radically nonrepresentational version of our model? The way the code works is that when `Representational` is set to 1 (that is, when the simulator is in Representational mode), there is a procedure where each duck chooses its preferred source based on food prevalence. Then, it 'aims' for the closest morsel of uneaten food at its preferred source. 

Well, what would happen if we just skipped the 'choose preferred source' step, and each duck just aimed for the closest morsel of food, no matter which source it is near? Would we still be able to observe probability matching? 

5. Click the `Representational` button to set representational mode to 0. This will skip the 'choose the preferred source' step. Confirm that you see 'Representational: 0' above the plot. No need to answer this question.
6. Try a few different rate combinations, keeping both sizes 8. What happens if you set both rates to the same small value, medium value, or large value? What happens if you make the ratio 2:1 (rate1 twice as large as rate2)?
7. Try a few different size combinations, initially setting both rates to 4. Explore and report a few observations.
8. With one interesting combination you found for #1 or #2 (or one or a few from each), see what happens as you make the pond taller. You may find the `Plot size` slider helpful to make the grid fit nicely in your browser window.
9. Draw some general conclusions. This simulation is using the 'representational' approach, where each duck tracks food density over time (size units per time units). Is it able to simulate probability matching due to differences in rate? Is it able to simulate probability matching due to differences in size?

# Lab report, part 3

Your answers for #2 and #7 should be different. Hint: you should have trouble seeing probability matching based on size when Representational is set to 0. 

10. Why do the Representational and Nonrepresentational modes differ for size?
11. Before looking ahead, can you think of any way you could make both modes capable of simulating probability matching based on size differences?

Now keep Representational = 0 and set "Food takes time" to 1. When 'Food takes time' is set to 0, food takes 1 time step for ducks to consume, no matter what the magnitude (size) value is. (You could change this to a different fixed value by searching for `time_to_consume = 1` and set it to a different value.) When 'Food takes time' is set to 1, it takes a duck `size` timesteps to eat a morsel of food (e.g., if `size1` is set to 8, when a duck reaches that morsel, it will stay stationary for 8 time steps and then it will be free to seek more food). 

12. How might setting 'Food takes time' to 1 change aspects of the simulation?
13. With 'Representational' set to 0 and 'Food takes time' set to 1, repeat your simulations with rate differences and size differences. What changes do you observe for size differences? Is it more like what you observed for #2 or for #7? Can you explain why?
14. Now set 'Representational' to 1 (and 'Food takes time' to 1), and repeat your simulations with size differences. What changes do you observe for size differences compared to #2? Is it better, worse, or just the same? Can you explain why?

# Lab report, part 4 -- challenge questions 

`Undergrads not registered for honors credit can skip this part if they want, or complete for about 2% extra credit`

**Choose at least one but do as many as you like!**

15. *Simulation challenge.* Examine how number of ducks relates to rate and magnitude. With the default parameters, what happens if you have only very few ducks or many more than the default? Can you describe 'boundary conditions' on the simuation -- that is, are there particular (approximate) ratios of number ducks and food prevalence that could help you estimate how many ducks you need at minimum to observe probability matching (for representational and/or nonrepresentational mode)?
16. *Simulation challenge.* Another possibility might be that you would find that ducks probability match in the  representational mode even when there is 'too much' or 'too little' food for probability matching to emerge with the nonrepresentational mode. What kinds of testable predictions (i.e., predictions you could test with real ducks) might that suggest? 
17. *Programming / theoretical challenge.* Examine the code where switchiness comes into play. Does this seem like an adequate analog to what Gallistel describes? (If your answer is simply 'yes', that's fine, but then don't bother with this question!) If not, can you propose an alternative way to implement switchiness? Double bonus: modify the code to implement your proposal and compare it to the original code (to do this, save a copy of the notebook with a new name where you can introduce your modifications).
18. *Pure programming challenge.* Can you fix the bug where many/all ducks suddenly change to red or violet and the change back? *Warning*: I have actually looked into this extensively and posted about it on matplotlib and python message boards and have had no success -- so this may not be solvable (e.g., if it is due to a bug in matplotlib).
19. *Pure programming challenge.* This simulator would be much more useful if it saved outputs that could be analyzed later. Can you come up with a way to create data files when you run simulations? Note that you will likely want to write the parameter settings to the file *or* make them part of the filename -- but you need some way to link the data file to the parameters.  


